**Başlamadan önce Çalışma Zaman Ayarlarından GPU seç**

# Adım 1: Kurulum

In [ ]:
!git clone https://github.com/tensorflow/models.git

In [ ]:
cd /content/models/research

In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
!git clone https://github.com/cocodataset/cocoapi.git

In [ ]:
cd /content/models/research/cocoapi/PythonAPI

In [ ]:
!make

In [ ]:
cp -r pycocotools /content/models/research

In [ ]:
cd /content/models/research

In [ ]:
cp object_detection/packages/tf2/setup.py .

In [ ]:
!python -m pip install --use-feature=2020-resolver .

# Adım 2: Kurulumu Test Et!

Aşağıdaki kod yürütüldüğünde en sonda aşağıdakine benzer bi çıktı elde etmen gerek
```
Ran 24 tests in 26.390s

OK (skipped=1)
```

In [ ]:
!python object_detection/builders/model_builder_tf2_test.py

# Adım 3: Çalışma Alanını Hazırlama

In [ ]:
cd /content

In [ ]:
mkdir workspace

In [ ]:
cd /content/workspace

In [ ]:
mkdir training_demo

In [ ]:
cd /content/workspace/training_demo

In [ ]:
mkdir annotations

In [ ]:
mkdir exported-models

In [ ]:
mkdir images

In [ ]:
mkdir models

In [ ]:
mkdir pre-trained-models

In [ ]:
cd /content/workspace/training_demo/images

In [ ]:
mkdir test

In [ ]:
mkdir train

In [ ]:
cd /content

In [ ]:
mkdir scripts

In [ ]:
cd /content/scripts

In [ ]:
mkdir preprocessing

In [ ]:
cd /content/scripts/preprocessing

# Adım 4: Data Hazırlama

**4.1. İlk önce labelimg ile resimleri etiketleme işlemini yaparak `xml` formatına çevirmen gerek.**

**4.2. Çevirilen xml formatlarını ve resimleri `/content/workspace/training_demo/images/train` ve `/content/workspace/training_demo/images/test` dizinlerine atman gerek.**

**4.3. `/content/workspace/training_demo/annotations` klasöründe `label_map.pbtxt` dosyası oluştur. İçerisinde şuna benzer bir yapı olmalı**
```
item {
    id: 1
    name: 'cat'
}

item {
    id: 2
    name: 'dog'
}
```

**4.4. [Buradan](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py) `generate_tfrecord.py` dosyasını indirin ve `/content/scripts/preprocessing` klasörünün içine atın**

In [ ]:
!python generate_tfrecord.py -x /content/workspace/training_demo/images/train -l /content/workspace/training_demo/annotations/label_map.pbtxt -o /content/workspace/training_demo/annotations/train.record

In [ ]:
!python generate_tfrecord.py -x /content/workspace/training_demo/images/test -l /content/workspace/training_demo/annotations/label_map.pbtxt -o /content/workspace/training_demo/annotations/test.record

NOT: Oluşturulan `.record` dosyaları 0 Byte olmamalıdır. Eğer 0 Byte ise eğitim düzgün çalışmayacaktır.

# Adım 5: Model Yükleme

In [ ]:
cd /content/workspace/training_demo/pre-trained-models

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz

In [ ]:
!tar -xvf ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz

In [ ]:
cd /content/workspace/training_demo/models

In [ ]:
mkdir my_ssd_resnet50_v1_fpn

**5.1. İndirilen modelin içinde bulunan `/content/workspace/training_demo/pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config` dosyasını, `/content/workspace/training_demo/models/my_ssd_resnet50_v1_fpn` de bulunan modelin içine kopyalayın**

**5.2. Kopyaladığınız `pipeline.config` dosyasını [buradan](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#configure-the-training-pipeline) düzenleyin.**

# Adım 6: Son Ayarlar

**6.1. `/content/models/research/object_detection/model_main_tf2.py` klasöründe bulunan `model_main_tf2.py` dosyasınını `/content/workspace/training_demo` içine kopyalayın.**

In [ ]:
!pip uninstall opencv-python

In [ ]:
!pip install opencv-python

In [ ]:
cd /content/workspace/training_demo

# Adım 7: Eğitim

**Eğitim uzun sürebilir. Sadece bekle!**

In [ ]:
!python model_main_tf2.py --model_dir=models/my_ssd_resnet50_v1_fpn --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/pipeline.config

# Adım 8: Modeli Kaydetme

**8.1. `/content/models/research/object_detection/exporter_main_v2.py` klasöründe bulunan `exporter_main_v2.py` dosyasını `/content/workspace/training_demo` içine kopyalayın**

In [ ]:
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path /content/workspace/training_demo/models/my_ssd_resnet50_v1_fpn/pipeline.config --trained_checkpoint_dir /content/workspace/training_demo/models/my_ssd_resnet50_v1_fpn --output_directory /content/workspace/training_demo/exported-models/my_model

# Adım 9: Modeli Test Etme

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
#import pathlib
import tensorflow as tf
import cv2
#import argparse

gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

#IMG_NAME = "drawing(76)"

IMAGE_PATHS = "./content/workspace/training_demo/images/test/drawing(76).png"

PATH_TO_MODEL_DIR = "./content/workspace/training_demo/exported-models/my_model"

PATH_TO_LABELS = "./content/workspace/training_demo/annotations/label_map.pbtxt"

#MIN_CONF_THRESH = float(0.60)

import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL = "./content/workspace/training_demo/exported-models/my_model/saved_model"

print("Loading model...", end="")
start_time = time.time()

detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Done! Took {elapsed_time} seconds")

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

import numpy as np
from PIL import Image
#import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

def load_iamge_into_numpy_array(path):
    return np.array(Image.open(path))

print(f"Running inference for {IMAGE_PATHS}...", end="")

image = cv2.imread(IMAGE_PATHS)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_expanded = np.expand_dims(image_rgb, axis=0)

input_tensor = tf.convert_to_tensor(image)
input_tensor = input_tensor[tf.newaxis, ...]
detections = detect_fn(input_tensor)

num_detections = int(detections.pop("num_detections"))
detections = {key: value[0, :num_detections ].numpy()
            for key, value in detections.items()}
detections["num_detections"] = num_detections

detections["detection_classes"] = detections["detection_classes"].astype(np.int64)

image_with_detections = image.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
    image_with_detections,
    detections['detection_boxes'],
    detections['detection_classes'],
    detections['detection_scores'],
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=200,
    min_score_thresh=0.5,
    agnostic_mode=False
)

print("Done")
cv2.imshow("Output", image_with_detections)
cv2.waitKey(0)

# Adım 10: Modeli Bilgisayara İndirme

In [ ]:
!zip -r /content/annotations.zip /content/workspace/training_demo/annotations

In [ ]:
!zip -r /content/exported-models.zip /content/workspace/training_demo/exported-models

In [ ]:
from google.colab import files
files.download("/content/annotations.zip")
files.download("/content/exported-models.zip")
